In [2]:
from simple_parsing import ArgumentParser
from pathlib import Path
import numpy as np
import os
from distribution_inference.datasets.utils import get_dataset_wrapper, get_dataset_information
from distribution_inference.attacks.utils import get_dfs_for_victim_and_adv, get_train_config_for_adv
from distribution_inference.attacks.whitebox.utils import wrap_into_loader, get_attack, get_train_val_from_pool
from distribution_inference.config import DatasetConfig, AttackConfig, WhiteBoxAttackConfig, TrainConfig
from distribution_inference.utils import flash_utils, warning_string
from distribution_inference.logging.core import AttackResult
from distribution_inference.attacks.whitebox.affinity.utils import get_seed_data_loader, identify_relevant_points, make_ds_and_loader


09/25/2022 00:26:03:WARNING:The OGB package is out of date. Your version is 1.3.3, while the latest version is 1.3.4.


In [7]:
parser = ArgumentParser(add_help=False)
parser.add_argument(
    "--load_config", help="Specify config file",
    type=Path, required=True)
parser.add_argument(
    '--gpu',
    default=None, help="device number")
parser.add_argument(
    "--ratios",
    nargs='+',
    type=float,default = None)
parser.add_argument(
    "--trial",
    type=int,
    default=None)
args = parser.parse_args(["--load_config","bone_gender_aga.json"])

In [8]:
attack_config: AttackConfig = AttackConfig.load(
    args.load_config, drop_extra_fields=False)
if args.gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
# Extract configuration information from config file
wb_attack_config: WhiteBoxAttackConfig = attack_config.white_box
train_config: TrainConfig = attack_config.train_config
data_config: DatasetConfig = train_config.data_config
if train_config.misc_config is not None:
    # TODO: Figure out best place to have this logic in the module
    if train_config.misc_config.adv_config:
        # Scale epsilon by 255 if requested
        if train_config.misc_config.adv_config.scale_by_255:
            train_config.misc_config.adv_config.epsilon /= 255
# Do the same if adv_misc_config is present
if attack_config.adv_misc_config is not None:
    if attack_config.adv_misc_config.adv_config:
        if attack_config.adv_misc_config.adv_config.scale_by_255:
            attack_config.adv_misc_config.adv_config.epsilon /= 255

# Make sure regression config is not being used here
if wb_attack_config.regression_config:
    raise ValueError(
        "This script is not designed to be used with regression attacks")


In [9]:
# Print out arguments
flash_utils(attack_config)

# Get dataset wrapper
ds_wrapper_class = get_dataset_wrapper(data_config.name)

# Get dataset info object
ds_info = get_dataset_information(data_config.name)()

# Create new DS object for both and victim
data_config_adv_1, data_config_victim_1 = get_dfs_for_victim_and_adv(
    data_config)
ds_vic_1 = ds_wrapper_class(
    data_config_victim_1, skip_data=not attack_config.victim_local_attack,label_noise=train_config.label_noise)
if not attack_config.victim_local_attack:
    ds_adv_1 = ds_wrapper_class(data_config_adv_1)

# Make train config for adversarial models
train_config_adv = get_train_config_for_adv(train_config, attack_config)


==> Arguments:
train_config  : 
    data_config  : 
        name  :  boneage
        prop  :  gender
        value  :  0.5
        split  :  adv
        drop_senstive_cols  :  False
        scale  :  1.0
        augment  :  False
        classify  :  age
        cwise_samples  :  None
        squeeze  :  True
        processed_variant  :  True
        prune  :  0.0
    epochs  :  0
    learning_rate  :  0.0
    batch_size  :  0
    misc_config  :  None
    lr_scheduler  :  None
    verbose  :  False
    num_models  :  1
    offset  :  0
    weight_decay  :  0.0
    get_best  :  True
    cpu  :  False
    expect_extra  :  True
    save_every_epoch  :  False
    extra_info  :  None
    regression  :  False
    multi_class  :  False
    label_noise  :  0.0
    model_arch  :  bonemodel
    parallel  :  False
values  :  [0.4, 0.6]
black_box  :  None
white_box  : 
    attack  :  affinity
    epochs  :  300
    batch_size  :  500
    learning_rate  :  0.0001
    weight_decay  :  0.01
    trai

In [10]:
# Load victim models for first value
models_vic_1 = ds_vic_1.get_models(
    train_config,
    n_models=attack_config.num_victim_models,
    on_cpu=attack_config.on_cpu,
    shuffle=False,
    model_arch=attack_config.victim_model_arch)

Available models: 250


Loading models: 100%|██████████| 250/250 [00:17<00:00, 13.90it/s]


In [11]:
prop_value = attack_config.values[0]

In [12]:
data_config_adv_2, data_config_vic_2 = get_dfs_for_victim_and_adv(
    data_config, prop_value=prop_value)

# Create new DS object for both and victim (for other ratio)
ds_vic_2 = ds_wrapper_class(
    data_config_vic_2,
    skip_data=not attack_config.victim_local_attack,
    label_noise=train_config.label_noise)
if not attack_config.victim_local_attack:
    ds_adv_2 = ds_wrapper_class(data_config_adv_2)

# Load victim's model features for other value
models_vic_2 = ds_vic_2.get_models(
    train_config,
    n_models=attack_config.num_victim_models,
    on_cpu=attack_config.on_cpu,
    shuffle=False,
    model_arch=attack_config.victim_model_arch)


Available models: 250


Loading models: 100%|██████████| 250/250 [00:20<00:00, 12.21it/s]


In [14]:
if not attack_config.victim_local_attack:
    test_data = wrap_into_loader(
        [models_vic_1, models_vic_2],
        batch_size=wb_attack_config.batch_size,
        shuffle=False,
        wrap_with_loader=False
    )

# Load adv models for both ratios
# Unless victim-only mode
if not attack_config.victim_local_attack:
    models_adv_1 = ds_adv_1.get_models(
        train_config_adv,
        n_models=attack_config.num_total_adv_models,
        on_cpu=attack_config.on_cpu,
        shuffle=False,
        model_arch=attack_config.adv_model_arch)

    models_adv_2 = ds_adv_2.get_models(
        train_config_adv,
        n_models=attack_config.num_total_adv_models,
        on_cpu=attack_config.on_cpu,
        shuffle=False,
        model_arch=attack_config.adv_model_arch)

Available models: 1000


Loading models: 100%|██████████| 1000/1000 [01:09<00:00, 14.43it/s]


Available models: 1000


Loading models: 100%|██████████| 1000/1000 [00:59<00:00, 16.72it/s]


In [ ]:
if attack_config.victim_local_attack:
    # Split victim models into train-val
    train_data, val_data = get_train_val_from_pool(
        [models_vic_1, models_vic_2],
        wb_config=wb_attack_config,
        wrap_with_loader=False
    )
    test_data = val_data
else:
    # Normal train-val split from adv models
    train_data, val_data = get_train_val_from_pool(
        [models_adv_1, models_adv_2],
        wb_config=wb_attack_config,
        wrap_with_loader=False
    )

In [ ]:
args = parser.parse_args(["--load_config","bone_gender_aga.json"])
attack_config: AttackConfig = AttackConfig.load(
    args.load_config, drop_extra_fields=False)
if args.gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
# Extract configuration information from config file
wb_attack_config: WhiteBoxAttackConfig = attack_config.white_box
train_config: TrainConfig = attack_config.train_config
data_config: DatasetConfig = train_config.data_config
if train_config.misc_config is not None:
    # TODO: Figure out best place to have this logic in the module
    if train_config.misc_config.adv_config:
        # Scale epsilon by 255 if requested
        if train_config.misc_config.adv_config.scale_by_255:
            train_config.misc_config.adv_config.epsilon /= 255
# Do the same if adv_misc_config is present
if attack_config.adv_misc_config is not None:
    if attack_config.adv_misc_config.adv_config:
        if attack_config.adv_misc_config.adv_config.scale_by_255:
            attack_config.adv_misc_config.adv_config.epsilon /= 255

# Make sure regression config is not being used here
if wb_attack_config.regression_config:
    raise ValueError(
        "This script is not designed to be used with regression attacks")
